<a href="https://colab.research.google.com/github/divyanshsaxena21/Predict_Podcast_Listening_Time/blob/main/Podcast_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
import zipfile

with zipfile.ZipFile('/content/playground-series-s5e4.zip', 'r') as zip_ref:
    zip_ref.extractall('extracted_files')


In [ ]:
df = pd.read_csv('/content/extracted_files/train.csv')

In [ ]:
df.columns

Index(['id', 'Podcast_Name', 'Episode_Title', 'Episode_Length_minutes',
       'Genre', 'Host_Popularity_percentage', 'Publication_Day',
       'Publication_Time', 'Guest_Popularity_percentage', 'Number_of_Ads',
       'Episode_Sentiment', 'Listening_Time_minutes'],
      dtype='object')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 12 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           750000 non-null  int64  
 1   Podcast_Name                 750000 non-null  object 
 2   Episode_Title                750000 non-null  object 
 3   Episode_Length_minutes       662907 non-null  float64
 4   Genre                        750000 non-null  object 
 5   Host_Popularity_percentage   750000 non-null  float64
 6   Publication_Day              750000 non-null  object 
 7   Publication_Time             750000 non-null  object 
 8   Guest_Popularity_percentage  603970 non-null  float64
 9   Number_of_Ads                749999 non-null  float64
 10  Episode_Sentiment            750000 non-null  object 
 11  Listening_Time_minutes       750000 non-null  float64
dtypes: float64(5), int64(1), object(6)
memory usage: 68.7+ MB


In [ ]:
# Define target and features
X = df.drop(columns=['Listening_Time_minutes', 'id'])
y = df['Listening_Time_minutes']

In [ ]:
# Handle missing values in numerical columns
numerical_features = ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Number_of_Ads']
categorical_features = ['Podcast_Name', 'Episode_Title', 'Genre', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']


In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Define preprocessing pipeline
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Impute missing values with median
    ('scaler', StandardScaler())  # Scale numerical data
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),  # Handle missing categorical data
    ('encoder', OneHotEncoder(handle_unknown='ignore'))  # One-hot encode categorical features
])

# Combine preprocessing steps for numerical and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])


In [ ]:
# Step 2: Train LightGBM model
lgbm_model = lgb.LGBMRegressor(n_estimators=100, random_state=42)
lgbm_model.fit(X_train, y_train)


ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: Podcast_Name: object, Episode_Title: object, Genre: object, Publication_Day: object, Publication_Time: object, Episode_Sentiment: object

In [ ]:
# Calculate metrics on the training set
mse_train = mean_squared_error(y_train, train_predictions)
rmse_train = np.sqrt(mse_train)
mae_train = mean_absolute_error(y_train, train_predictions)
r2_train = r2_score(y_train, train_predictions)

# Display the evaluation metrics
print("Training Set Evaluation Metrics:")
print(f"R² (R-squared): {r2_train:.4f}")
print(f"RMSE (Root Mean Squared Error): {rmse_train:.4f}")
print(f"MAE (Mean Absolute Error): {mae_train:.4f}")
print(f"MSE (Mean Squared Error): {mse_train:.4f}")

Training Set Evaluation Metrics:
R² (R-squared): 0.7735
RMSE (Root Mean Squared Error): 12.9183
MAE (Mean Absolute Error): 9.4292
MSE (Mean Squared Error): 166.8827


In [ ]:
test_df = pd.read_csv('/content/extracted_files/test.csv')

In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 11 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           250000 non-null  int64  
 1   Podcast_Name                 250000 non-null  object 
 2   Episode_Title                250000 non-null  object 
 3   Episode_Length_minutes       221264 non-null  float64
 4   Genre                        250000 non-null  object 
 5   Host_Popularity_percentage   250000 non-null  float64
 6   Publication_Day              250000 non-null  object 
 7   Publication_Time             250000 non-null  object 
 8   Guest_Popularity_percentage  201168 non-null  float64
 9   Number_of_Ads                250000 non-null  float64
 10  Episode_Sentiment            250000 non-null  object 
dtypes: float64(4), int64(1), object(6)
memory usage: 21.0+ MB


In [ ]:
# Preprocess the test data (same preprocessing steps as for the training data)
X_test = test_df.drop(columns=['id'])  # Drop the 'id' column from the test set


In [ ]:
# Predict on the test set
test_predictions = model_pipeline.predict(X_test)


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [ ]:
# Prepare the submission DataFrame
submission = pd.DataFrame({
    'id': test_df['id'],  # The 'id' column from the test set
    'Listening_Time_minutes': test_predictions  # The predicted values
})

In [ ]:
# Save the submission file
submission.to_csv('submission_rf_lgbm .csv', index=False)
print("Submission file has been saved as 'submission.csv'.")

Submission file has been saved as 'submission.csv'.
